In [1]:
# Librairie Os pour les chemins de fichiers et variables d'environnement
import os
# Librairie csv pour créer et lire le format csv
import csv
# Lien au fichier Python pour exécuter ses fonctions
import functions_PgSql
# import Donnees
import psycopg2
import pandas as pd
import numpy as np

 
# Spécifier le nom de la base de données créée au préalable dans PgAdmin
ma_base_donnees = "Musees_V3"
utilisateur = "postgres"

# Méthode os.environ.get pour utiliser le mot de passe 
# enregistré au préalable dans une variable d'environnement
mot_passe = os.environ.get('pg_psw')

# Fonction conn appelle la fonction ouvrir_connexion du fichier annexe Python
conn = functions_PgSql.ouvrir_connection(ma_base_donnees, utilisateur, mot_passe)


## VUES SQL

In [2]:
# Création vue Oeuvres avec infos Artistes et Titres d'oeuvres
creation_vue_Oeuvres_Artistes = """
        CREATE VIEW v_Oeuvres_Artistes (
        Nom_Artiste,
        Id_Oeuvre,
        Titre_Oeuvre,
        Denomination_Oeuvre,
        Materieux_Techniques_Oeuvre,
        Sujet_Represente,
        Periode_Oeuvre,
        Id_Museo
        )
    AS
        SELECT a.nom_artiste, 
               o.id_oeuvre,
               o.titre_oeuvre, 
               o.denomination, 
               o.materiaux_techniques, 
               o.sujet_represente, 
               o.periode_creation, 
               o.id_museo
        FROM oeuvre o, oeuvre_artiste oa, artiste a
        
        WHERE o.id_oeuvre = oa.id_oeuvre
        AND oa.id_artiste = a.id_artiste;
"""

In [3]:
functions_PgSql.executer_requete(creation_vue_Oeuvres_Artistes, conn)

In [4]:
artistes_oeuvres = pd.read_sql_query("SELECT * FROM v_Oeuvres_Artistes WHERE nom_artiste LIKE '%RUBENS%' ORDER BY id_museo;", conn)

In [5]:
artistes_oeuvres.head()

,nom_artiste,id_oeuvre,titre_oeuvre,denomination_oeuvre,materieux_techniques_oeuvre,sujet_represente,periode_oeuvre,id_museo
0,RUBENS PETRUS PAULUS,70309,SAINTE FAMILLE AVEC SAINTE ELISABETH ET SAINT ...,Tableau,"peinture à l'huile, bois","scène biblique (Sainte Famille, Vierge, Enfant...",17e siècle,M0019
1,RUBENS PETRUS PAULUS,70306,LA VISITATION,Tableau,"peinture à l'huile, bois","scène biblique (Visitation, sainte Elisabeth, ...",1er quart 17e siècle,M0019
2,RUBENS PETRUS PAULUS,70426,"LE CHRIST TRIOMPHANT DE LA MORT ET DU PECHE, S...",Tableau,"peinture à l'huile, bois","figure biblique (Salvator Mundi, Christ, en pi...",1er quart 17e siècle,M0019
3,RUBENS PETRUS PAULUS,70425,SAINT FRANCOIS D'ASSISE,Tableau,"peinture à l'huile, toile","figure (saint François d'Assise, en pied, mouton)",1ère moitié 17e siècle,M0019
4,RUBENS PIERRE PAUL,126953,LE MARTYRE DE SAINT GEORGES,NaN,NaN,NaN,NaN,M0065


In [6]:
# Création vue Oeuvres avec infos Catégories
creation_vue_Oeuvres_Categories = """
        CREATE VIEW v_Oeuvres_Categories (
        Nom_Categorie,
        Id_oeuvre,
        Titre_Oeuvre,
        Denomination_Oeuvre,
        Materieux_Techniques_Oeuvre,
        Sujet_Represente,
        Periode_Oeuvre,
        Id_Museo
        )
    AS
        SELECT c.domaine, 
               o.id_oeuvre,
               o.titre_oeuvre, 
               o.denomination, 
               o.materiaux_techniques, 
               o.sujet_represente, 
               o.periode_creation, 
               o.id_museo
        FROM oeuvre o, oeuvre_categorie oc, categorie c
        
        WHERE o.id_oeuvre = oc.id_oeuvre
        AND oc.id_categorie = c.id_categorie;
"""

In [7]:
functions_PgSql.executer_requete(creation_vue_Oeuvres_Categories, conn)

In [8]:
oeuvres_categories = pd.read_sql_query("SELECT * FROM v_Oeuvres_Categories WHERE Nom_Categorie LIKE '%%';", conn)

In [9]:
oeuvres_categories.head()

,nom_categorie,id_oeuvre,titre_oeuvre,denomination_oeuvre,materieux_techniques_oeuvre,sujet_represente,periode_oeuvre,id_museo
0,SCULPTURE,0,VIERGE A L'ENFANT,Statue,"marbre, taille directe,revers plat, poli, dorure","figures bibliques (Vierge à l'Enfant, en pied,...",1ère moitié 16e siècle,M5012
1,SCULPTURE,1,"LA TRINITE, COMPASSION DU PERE",Groupe relié,"modelage,revers partiellement évidé,terre cuit...",figures bibliques (Trinité),1er quart 17e siècle,M5012
2,SCULPTURE,2,SAINTE SAVINE,Statue,"taille directe,revers travaillé,pierre","figure (sainte Savine, en pied, livre, bijou, ...",16e siècle,M5012
3,SCULPTURE,3,"L'ASTRONOMIE, LA SYBILLE DE CUMES (?)",Statue,pierre,"figure (allégorie : astronomie, femme, en pied...",16e siècle,M5012
4,SCULPTURE,4,LETTRE F,"Image ouvrante,relief","Bas-relief,taille en réserve,bois","scène biblique (Calvaire, en médaillon),figure...",1er quart 16e siècle,M5012


In [10]:
# Infos artistes et titres des oeuvres
# Join table de relation many to many oeuvre_artiste
requete = '''
    SELECT o.titre_oeuvre, a.*
    FROM oeuvre o
    
    RIGHT JOIN oeuvre_artiste oa
        ON o.id_oeuvre = oa.id_oeuvre 
    RIGHT JOIN artiste a
        ON oa.id_artiste = a.id_artiste
    
    ORDER BY titre_oeuvre;
'''
Info_Artiste_Titre = pd.read_sql_query(requete, conn)
print(Info_Artiste_Titre.shape)
Info_Artiste_Titre.head()

(479429, 5)


,titre_oeuvre,id_artiste,nom_artiste,precision_auteur,ecole_pays
0,' IRIS ' - PROJET POUR UNE SCULPTURE SERIELLE ...,1079,DESCOMBIN MAXIME,"Le puley, 1909, charnay-lès-mâcon, 2003",FRANCE
1,' QUELLE FOLIE DE PENSER ENCORE AU MARIAGE ! ',13629,GOYA Y LUCIENTES FRANCISCO DE,"fuendetodos, 1746, bordeaux, 1828, nationalité...",ESPAGNE
2,' VEILLEUSE ' - ETUDE (TITRE INSCRIT SUR L'OEU...,1079,DESCOMBIN MAXIME,"Le puley, 1909, charnay-lès-mâcon, 2003",FRANCE
3,'69',5378,HAUSMANN RAOUL,"Vienne, 1886, limoges, 1971",AUTRICHE
4,'69',5378,HAUSMANN RAOUL,"Vienne, 1886, limoges, 1971",AUTRICHE


In [11]:
# Titre et nom artiste
requete = '''
    SELECT titre_oeuvre, nom_artiste
    FROM oeuvre o
    
    JOIN oeuvre_artiste oa
        ON o.id_oeuvre = oa.id_oeuvre 
    JOIN artiste a
        ON oa.id_artiste = a.id_artiste
    
    ORDER BY titre_oeuvre;
'''
Titre_Artiste = pd.read_sql_query(requete, conn)
print(Titre_Artiste.shape)
Titre_Artiste.head()

(479429, 2)


,titre_oeuvre,nom_artiste
0,' IRIS ' - PROJET POUR UNE SCULPTURE SERIELLE ...,DESCOMBIN MAXIME
1,' QUELLE FOLIE DE PENSER ENCORE AU MARIAGE ! ',GOYA Y LUCIENTES FRANCISCO DE
2,' VEILLEUSE ' - ETUDE (TITRE INSCRIT SUR L'OEU...,DESCOMBIN MAXIME
3,'69',HAUSMANN RAOUL
4,'69',HAUSMANN RAOUL


In [12]:
# Nombre d'oeuvres réalisées par artistes
# Recherche par nom
requete = '''
    SELECT DISTINCT nom_artiste, 
           COUNT(o.id_oeuvre) AS Nb_Oeuvre
    FROM oeuvre o
    
    JOIN oeuvre_artiste oa
        ON o.id_oeuvre = oa.id_oeuvre 
    JOIN artiste a
        ON oa.id_artiste = a.id_artiste
    
    WHERE nom_artiste <> 'ANONYME'
    
    GROUP BY nom_artiste
    
    HAVING COUNT(o.id_oeuvre) >= 100
    
    ORDER BY nb_oeuvre DESC;
'''
Nb_Oeuvres_Artiste = pd.read_sql_query(requete, conn)
print(Nb_Oeuvres_Artiste.shape)
Nb_Oeuvres_Artiste.head()

(543, 2)


,nom_artiste,nb_oeuvre
0,INCONNU,18140
1,MANUFACTURE DE SEVRES,17560
2,CHAPU HENRI MICHEL ANTOINE,9811
3,RODIN AUGUSTE,9426
4,BOUDIN EUGENE,7871


In [13]:
# Quels Artistes exposés par Ville
# Recherche par Ville, Infos Musées et noms artistes
# Utilisation de la vue v_Oeuvres_Artistes avec une autre jointure
requete = """
    SELECT DISTINCT a.Nom_Artiste,
           COUNT(a.id_oeuvre) AS Nb_Oeuvres,
           m.nom_officiel AS Nom_Musee,
           m.ville
    FROM v_Oeuvres_Artistes a
    
    JOIN musee m 
        ON a.Id_Museo = m.Id_Museo
    
    WHERE m.ville = 'Nantes'
    GROUP BY a.Nom_Artiste, m.nom_officiel, m.ville
    ORDER BY Nom_Musee, Nb_Oeuvres DESC;
"""
Localiser_Ville = pd.read_sql_query(requete, conn)
print(Localiser_Ville.shape)
Localiser_Ville.head()

(1437, 4)


,nom_artiste,nb_oeuvres,nom_musee,ville
0,ANONYME,587,musée des Beaux-Arts,Nantes
1,MERSON LUC OLIVIER,163,musée des Beaux-Arts,Nantes
2,DELAUNAY JULES ELIE,118,musée des Beaux-Arts,Nantes
3,GORIN JEAN,105,musée des Beaux-Arts,Nantes
4,GORIN JEAN ALBERT,105,musée des Beaux-Arts,Nantes


In [14]:
# Quels Musées Exposent un Artiste
# Recherche par Nom Artiste, Infos Musées et noms artistes
# Utilisation de la vue v_Oeuvres_Artistes avec une autre jointure
requete = """
    SELECT DISTINCT a.Nom_Artiste,
           m.nom_officiel AS Nom_Musee,
           m.ville
    FROM v_Oeuvres_Artistes a
    
    JOIN musee m 
        ON a.Id_Museo = m.Id_Museo
    
    WHERE  a.Nom_Artiste LIKE '%RENOI%'
    
    ORDER BY Nom_Musee, ville;
"""
Localiser_Artiste = pd.read_sql_query(requete, conn)
print(Localiser_Artiste.shape)
Localiser_Artiste.head()

(24, 3)


,nom_artiste,nom_musee,ville
0,RENOIR PIERRE AUGUSTE,Château-musée,Dieppe
1,RENOIR PIERRE AUGUSTE,Domaine National de Versailles - Le château,Versailles
2,RENOIR PIERRE AUGUSTE,Musée d'Art Moderne André Malraux - MuMa,Le Havre
3,RENOIR AUGUSTE,musée d'art moderne et contemporain de Strasbourg,Strasbourg
4,RENOIR PIERRE AUGUSTE,musée d'art moderne et contemporain de Strasbourg,Strasbourg


In [15]:
# Quels Musées Exposent un Artiste et nombre d'oeuvres par Artistes
# Recherche par Nom Artiste
# Utilisation de la vue v_Oeuvres_Artistes avec une autre jointure
requete = """
    SELECT a.Nom_Artiste, 
           COUNT(DISTINCT a.id_oeuvre) AS Nb_Oeuvres,
           m.nom_officiel AS Nom_Musee,
           m.ville
    FROM v_Oeuvres_Artistes a
    
    JOIN musee m 
        ON a.Id_Museo = m.Id_Museo
    
    WHERE  a.Nom_Artiste LIKE '%VINCI%'
    
    GROUP BY a.Nom_Artiste, m.nom_officiel, m.ville 
    
    ORDER BY nb_oeuvres DESC;
"""
Localiser_Artiste_Nb_Oeuvres = pd.read_sql_query(requete, conn)
print(Localiser_Artiste_Nb_Oeuvres.shape)
Localiser_Artiste_Nb_Oeuvres.head(38)

(38, 4)


,nom_artiste,nb_oeuvres,nom_musee,ville
0,VINCI LEONARDO DA,187,musée du Louvre,Paris
1,VINCIDOR TOMMASO,12,musée du Louvre,Paris
2,LEONARDO DI SER PIERO DA VINCI,10,musée du Louvre,Paris
3,VINCI LEONARD DE,8,musée du Louvre,Paris
4,LEONARD DE VINCI,7,musée Gustave Moreau,Paris
5,LEONARD DE VINCI,6,musée Ingres,Montauban
6,LEONARDO DI SER PIERO DA VINCI,5,musée Ingres,Montauban
7,VINCI LEONARD DE,5,musée Ingres,Montauban
8,LEONARD DE VINCI,4,musée des Beaux-Arts,Strasbourg
9,LEONARDO DI SER PIERO DA VINCI,3,musée des Beaux-Arts,Rennes


In [16]:
# Quels Musées Exposent un Artiste sur une période donnée
# Recherche par période : siècle, Infos Musées et noms artistes et nombre d'oeuvres exposées
# Utilisation de la vue v_Oeuvres_Artistes avec une autre jointure
requete = """
    SELECT a.Nom_Artiste, 
           COUNT(a.id_oeuvre) AS Nb_Oeuvres,
           a.periode_oeuvre,
           m.nom_officiel AS Nom_Musee,
           m.ville
    FROM v_Oeuvres_Artistes a
    
    JOIN musee m 
        ON a.Id_Museo = m.Id_Museo
    
    WHERE  a.Periode_Oeuvre LIKE '%21%'
    
    GROUP BY a.Nom_Artiste, 
             a.periode_oeuvre, 
             m.nom_officiel, 
             m.ville
    
    ORDER BY ville;
"""
Localiser_Periode = pd.read_sql_query(requete, conn)
print(Localiser_Periode.shape)
Localiser_Periode.head()

(484, 5)


,nom_artiste,nb_oeuvres,periode_oeuvre,nom_musee,ville
0,BLOCH PIERRETTE,1,1er quart 21e siècle,musée Picasso,Antibes
1,LESUEUR NATACHA,1,1er quart 21e siècle,musée Picasso,Antibes
2,SUNDKVIST JOHN,3,1er quart 21e siècle,musée Picasso,Antibes
3,VIALLAT CLAUDE,1,"2e moitié 20e siècle,1er quart 21e siècle",musée Picasso,Antibes
4,VILLERS ANDRE,1,"2e moitié 20e siècle,1er quart 21e siècle",musée Picasso,Antibes


In [17]:
# Poucentage d'oeuvres exposées par villes / total oeuvres
requete = """
    SELECT m.ville,
           COUNT(o.id_oeuvre) AS Total_Oeuvres,
           ROUND(ROUND(COUNT(o.id_oeuvre),2)*100/(SELECT ROUND(COUNT(o.id_oeuvre),2) FROM oeuvre o),2) || ' %' AS Part_Oeuvres,
           COUNT(DISTINCT o.Id_Museo) AS Nb_Musees
    FROM oeuvre o,
    	musee m
 	
    WHERE o.Id_Museo = m.Id_Museo
    
    GROUP BY ville
    ORDER BY total_oeuvres DESC;
"""
Part_Oeuvres = pd.read_sql_query(requete, conn)
print(Part_Oeuvres.shape)
Part_Oeuvres.head()

(296, 4)


,ville,total_oeuvres,part_oeuvres,nb_musees
0,Paris,166299,40.21 %,27
1,Sèvres,17632,4.26 %,1
2,Strasbourg,14795,3.58 %,7
3,Marseille,14532,3.51 %,7
4,Chantilly,9178,2.22 %,1


In [18]:
# Poucentage d'oeuvres exposées par région, total oeuvres et musées
requete = """
    SELECT r.nom_region,
           COUNT(DISTINCT o.id_museo) AS Nb_Musees,
           COUNT(o.id_oeuvre) AS Total_Oeuvres,
           ROUND(ROUND(COUNT(o.id_oeuvre),2)*100/(SELECT ROUND(COUNT(o.id_oeuvre),2) FROM oeuvre o),2) || ' %' AS Part_Oeuvres
    FROM oeuvre o,
    	 musee m,
         departement d,
         region r
 	
    WHERE o.Id_Museo = m.Id_Museo
    AND m.code_dpt = d.id_dpt
    AND d.id_region = r.id_region
    
    GROUP BY nom_region
    ORDER BY total_oeuvres DESC;
"""
Région_Oeuvres = pd.read_sql_query(requete, conn)
print(Région_Oeuvres.shape)
Région_Oeuvres.head(30)

(14, 4)


,nom_region,nb_musees,total_oeuvres,part_oeuvres
0,Ile-de-France,63,201572,48.74 %
1,Hauts-de-France,37,39527,9.56 %
2,Bourgogne-Franche-Comté,45,30916,7.48 %
3,Grand Est,45,28969,7.01 %
4,Provence-Alpes-Côte d'Azur,40,27121,6.56 %
5,Occitanie,28,17945,4.34 %
6,Pays-de-la-Loire,16,17277,4.18 %
7,Auvergne-Rhône-Alpes,30,11790,2.85 %
8,Nouvelle-Aquitaine,27,11611,2.81 %
9,Bretagne,14,11546,2.79 %


In [19]:
# Poucentage d'oeuvres exposées par départements, total oeuvres et musées
requete = """
    SELECT d.nom_departement,
           COUNT(DISTINCT o.id_museo) AS Nb_Musees,
           COUNT(o.id_oeuvre) AS Total_Oeuvres,
           ROUND(ROUND(COUNT(o.id_oeuvre),2)*100/(SELECT ROUND(COUNT(o.id_oeuvre),2) 
                                                  FROM oeuvre o),2) || ' %' AS Part_Oeuvres
    FROM oeuvre o,
    	 musee m,
         departement d
 	
    WHERE o.Id_Museo = m.Id_Museo
    AND m.code_dpt = d.id_dpt
    
    GROUP BY nom_departement
    ORDER BY total_oeuvres DESC;
"""
Dpt_Oeuvres = pd.read_sql_query(requete, conn)
print(Dpt_Oeuvres.shape)
Dpt_Oeuvres.head()

(93, 4)


,nom_departement,nb_musees,total_oeuvres,part_oeuvres
0,Paris,29,166989,40.38 %
1,Hauts-de-Seine,8,19985,4.83 %
2,Oise,10,19019,4.60 %
3,Bouches-du-Rhône,14,17206,4.16 %
4,Bas-Rhin,14,15140,3.66 %


In [20]:
# Poucentage d'oeuvres exposées par villes, total oeuvres et musées
requete = """
    SELECT m.ville,
           COUNT(DISTINCT o.id_museo) AS Nb_Musees,
           COUNT(o.id_oeuvre) AS Total_Oeuvres,
           ROUND(ROUND(COUNT(o.id_oeuvre),2)*100/(SELECT ROUND(COUNT(o.id_oeuvre),2) 
                                                  FROM oeuvre o),2) || ' %' AS Part_Oeuvres
    FROM oeuvre o,
    	 musee m
 	
    WHERE o.Id_Museo = m.Id_Museo
        
    GROUP BY ville
    ORDER BY nb_musees DESC;
"""
Villes_Oeuvres = pd.read_sql_query(requete, conn)
print(Villes_Oeuvres.shape)
Villes_Oeuvres.head(30)

(296, 4)


,ville,nb_musees,total_oeuvres,part_oeuvres
0,Paris,27,166299,40.21 %
1,Nice,10,7528,1.82 %
2,Rouen,8,4493,1.09 %
3,Marseille,7,14532,3.51 %
4,Toulouse,7,3443,0.83 %
5,Strasbourg,7,14795,3.58 %
6,Bordeaux,4,7812,1.89 %
7,Aix-en-Provence,3,14,0.00 %
8,Dijon,3,2209,0.53 %
9,Bourges,3,573,0.14 %


In [21]:
# Dates d'acquisition, nombre d'oeuvres
requete = """
    SELECT o.date_acquisition,
           COUNT(o.id_oeuvre) AS Total_Oeuvres,
           ROUND(ROUND(COUNT(o.id_oeuvre),2)*100/(SELECT ROUND(COUNT(o.id_oeuvre),2) FROM oeuvre o),2) || ' %' AS Part_Oeuvres,
           COUNT(DISTINCT o.Id_Museo) AS Nb_Musees
           
    FROM oeuvre o,
    	musee m
 	
    WHERE o.Id_Museo = m.Id_Museo
    AND date_acquisition != 'NaN'
    AND date_acquisition NOT LIKE '%acquisition%'

    GROUP BY date_acquisition
    ORDER BY total_oeuvres DESC;
"""
date_acquisition = pd.read_sql_query(requete, conn)
print(date_acquisition.shape)
date_acquisition.head()

(4518, 4)


,date_acquisition,total_oeuvres,part_oeuvres,nb_musees
0,2011,17581,4.25 %,45
1,1927,12508,3.02 %,61
2,1930,11741,2.84 %,67
3,1922,10248,2.48 %,62
4,1916,9196,2.22 %,30


In [22]:
# Poucentage d'oeuvres exposées par villes, total oeuvres et musées
# Filtre sur un artiste, modifier la sous requête SELECT ROUND et WHERE requête principale
requete = """
    SELECT m.ville,
           a.nom_artiste AS Nom_Artiste,
           COUNT(DISTINCT a.id_museo) AS Nb_Musees,
           COUNT(a.id_oeuvre) AS Total_Oeuvres,
           ROUND(ROUND(COUNT(a.id_oeuvre),2)*100/(SELECT ROUND(COUNT(a.id_oeuvre),2) 
                                                  FROM v_Oeuvres_Artistes a
                                                  WHERE a.nom_artiste LIKE '%RENOIR%'),2) || ' %' AS                                                   Part_Oeuvres
    FROM musee m,
         v_Oeuvres_Artistes a

    WHERE a.Id_Museo = m.Id_Museo
    AND a.nom_artiste LIKE '%RENOIR%'
    AND ville = 'Paris'
    GROUP BY ville, a.nom_artiste

    ORDER BY nb_musees DESC;
"""
Villes_Oeuvres_Artiste = pd.read_sql_query(requete, conn)
print(Villes_Oeuvres_Artiste.shape)
Villes_Oeuvres_Artiste.head(30)

(2, 5)


,ville,nom_artiste,nb_musees,total_oeuvres,part_oeuvres
0,Paris,RENOIR PIERRE AUGUSTE,4,119,73.91 %
1,Paris,RENOIR AUGUSTE,1,7,4.35 %


In [23]:
# Total oeuvres par domaine et musées
requete = """
    SELECT oc.Nom_Categorie,
           COUNT(oc.Id_oeuvre) AS Total_Oeuvres,
           COUNT(DISTINCT oc.id_museo) AS Nb_Musees
    FROM v_Oeuvres_Categories oc
 	
    GROUP BY Nom_Categorie
    ORDER BY nb_musees DESC;
"""
Categories_Oeuvres = pd.read_sql_query(requete, conn)
print(Categories_Oeuvres.shape)
Categories_Oeuvres.head()

(132, 3)


,nom_categorie,total_oeuvres,nb_musees
0,PEINTURE,69952,309
1,SCULPTURE,34055,287
2,DESSIN,243852,277
3,CÉRAMIQUE,30096,195
4,CROYANCES - COUTUMES,4091,148


In [24]:
# Total oeuvres par domaine et musées
# Par domaine et villes
# Recherche par ville et/ou domaine

requete = """
    SELECT m.ville,
           oc.Nom_Categorie,
           COUNT(oc.id_oeuvre) AS Total_Oeuvres,
           COUNT(DISTINCT oc.id_museo) AS Nb_Musees
    FROM v_Oeuvres_Categories oc
 	
    JOIN musee m
        ON oc.id_museo = m.id_museo

    WHERE ville LIKE '%Nantes%' AND Nom_categorie LIKE '%PEINTURE%'

    GROUP BY Nom_Categorie, m.ville;
"""
Categories_villes = pd.read_sql_query(requete, conn)
print(Categories_villes.shape)
Categories_villes.head()

(1, 4)


,ville,nom_categorie,total_oeuvres,nb_musees
0,Nantes,PEINTURE,2380,3


In [25]:
# Total oeuvres par domaine et musées
# Par domaine et villes
# Recherche par ville et/ou domaine

requete = """
            SELECT oc.nom_categorie,
            COUNT(oc.id_oeuvre) as Nombre_Oeuvres,
            COUNT(distinct oc.id_museo) as Nombre_Musees,
            m.geo_x,
            m.geo_y

        FROM v_Oeuvres_Categories oc

        JOIN musee m
                ON oc.id_museo = m.id_museo
        
            WHERE geo_x != 'Nan' AND geo_y != 'Nan' 
            AND oc.nom_categorie LIKE '%%'
            GROUP BY oc.nom_categorie, oc.nom_categorie, m.geo_x, m.geo_y;
"""
Categories_geo = pd.read_sql_query(requete, conn)
print(Categories_geo.shape)
Categories_geo.head()

(3466, 5)


,nom_categorie,nombre_oeuvres,nombre_musees,geo_x,geo_y
0,AFRIQUE,2,1,43.280400,5.380600
1,AFRIQUE,7,1,43.300172,5.367602
2,AFRIQUE,2,1,43.594189,1.448945
3,AFRIQUE,10,1,43.602258,1.443132
4,AFRIQUE,2,1,44.387609,6.655077


In [26]:
# Recherche doublons Domaine

requete = """
    SELECT c.domaine,
           COUNT(c.domaine) AS Nb_Domaine
    FROM categorie c
    GROUP BY c.domaine
    ORDER BY Nb_Domaine DESC;
"""
Categories_check = pd.read_sql_query(requete, conn)
print(Categories_check.shape)
Categories_check.head()

(135, 2)


,domaine,nb_domaine
0,ÉTRUSQUE,1
1,AFRIQUE DU NORD,1
2,ARTS GRAPHIQUES,1
3,SPECTACLE - FÀªTE,1
4,MODERNE,1


In [27]:
# Recherche doublons Oeuvre_Artiste

requete = """
    SELECT CONCAT(id_oeuvre, ' ', id_artiste) AS ids,
           COUNT(CONCAT(id_oeuvre, ' ', id_artiste)) AS Nb_ids
    FROM oeuvre_artiste
    GROUP BY ids
    ORDER BY Nb_ids DESC;
"""
OA_check = pd.read_sql_query(requete, conn)
print(OA_check.shape)
OA_check.head()

(479429, 2)


,ids,nb_ids
0,9227 62,1
1,8626 1859,1
2,2365 62,1
3,18011 1246,1
4,14290 2178,1


## SUPPRIMER VUE

In [ ]:
# Effacer une vue
#functions_PgSql.executer_requete("DROP VIEW IF EXISTS v_Oeuvres_Categories;", conn)